# Dubbing Task

This is an example Python notebook for running dubbing tasks using [Camb.ai](https://camb.ai)'s API

# Index
- [Dub Method](#dub-method)
- [Individual Methods](#individual-methods)
  - [Create End-to-End Dubbing Task](#create-end-to-end-dubbing-task)
  - [Get Dubbing Status](#get-dubbing-status)
  - [Get Dubbed Run Info](#get-dubbed-run-info)

In [1]:
# !pip install cambai

To get started, make sure you import both the `CambAI` class from the `cambai` module.

Ensure that you have the Camb AI API Key from https://studio.camb.ai

In [2]:
import os
from cambai import CambAI

In [4]:
client = CambAI(
    api_key=os.environ.get("CAMB_API_KEY") # Get your API key from https://studio.camb.ai
)

👆 While you can provide an `api_key` as an argument, we recommend using [`python-dotenv`](https://pypi.org/project/python-dotenv/) to add `CAMB_API_KEY="Your API Key"` to your `.env` file to prevent your API Key from being stored in source control.

---
To get the numbers for the list of all available languages, you can use the `get_languages()` method.

You can also save the list of languages to a file by setting the `write_to_file` argument to `True`.

In [ ]:
client.get_languages("source", write_to_file=True)
# client.get_languages("target", write_to_file=True)

## Dub Method

You can call the `dub` method to perform the entire end-to-end dubbing process.<br>
From calling the API, to getting the final dubbed run info.

In [ ]:
values = client.dub(
    video_url="https://youtu.be/rlK-BySadHk", # Any publicly accessible video URL
    source_language=37,  # English (Ireland)
    target_language=54,  # Spanish (Spain)
    debug=True,          # You can set this to True to see the progress of the dubbing process
    polling_interval=70
)

**NOTE:** The dubbing process can take a while to complete, so it's better to set the polling interval to a higher value, especially for longer videos.

In [ ]:
values["video_url"]

In [ ]:
values["audio_url"]

---
---

## Individual Methods

You can also access individual methods to have more control over the dubbing task.

- [`start_dubbbing()`](#create-end-to-end-dubbing-task)
- [`get_dubbing_status()`](#get-dubbing-status)
- [`get_dubbed_run_info()`](#get-dubbed-run-info)

### Create End-to-End Dubbing Task

This method creates the dubbing task and returns the `run_id` which can be used to check the status of the dubbing task.

In [6]:
task = client.start_dubbing("https://youtu.be/rlK-BySadHk", 37, 125)

task_id = task["task_id"]

In [7]:
task_id

'31f7af5f-346d-49c9-930b-f2d5e5dc5b4a'

---

### Get Dubbing Status

This method returns the status of the dubbing task.

If you got the "SUCCESS" status, then the dubbing task is completed and you can get the download link for the dubbed audio from `get_dubbed_run_info()`.

In [12]:
dubbing_status = client.get_dubbing_status(task_id)

status = dubbing_status["status"]
run_id = dubbing_status["run_id"]

In [13]:
status

'SUCCESS'

In [11]:
run_id

27897

**NOTE:** The dubbing task may take some time to complete.

----

### Get Dubbed Run Info

You can use this method to get the links for downloading the dubbed video and audio,<br>
as well as the transcript of the dubbed video in the target language.

In [14]:
dubbed_info = client.get_dubbed_run_info(run_id)

In [15]:
dubbed_info["video_url"]

'https://storage.googleapis.com/cambai-prod-bucket/temp/dubbed_runs/27897/output_video.mp4?Expires=1720547749&GoogleAccessId=cambai-deployment%40camb-ai-82e9a.iam.gserviceaccount.com&Signature=HR4Llr97rmqlUWPsmzVYB0aBgcasp%2BULl2UfTlJTJOJpOJHeouzUnQpfaEhEEiQib2cNcJy5ItSM1M1kxS8y%2FAiAmuWHVoAWWqfRoUobFwbAo2mMtG6XFhYrvTRsJfd8QDxQisUy7pnCe1TXCG%2B6YaH36cYuo5iYqfnTzLH8HK0TfEwfnzB7PmZR5ASDlZGwPsdxPZECNBDcT6wmnDxGDo67ifBgT3Syoe5SzihzKePrpBjbaj28qvz24eV60HeCOcfnk9TWpALqvtHlhOebgp%2BdQmN%2F%2FgdBciAWRq6%2FrBhMKADE3wPZsYZ%2FAnRQmwRVjc1PO8E9sgz2yxcOfFTUTw%3D%3D'

In [17]:
dubbed_info["audio_url"]

'https://storage.googleapis.com/cambai-prod-bucket/temp/dubbed_runs/27897/output_audio.wav?Expires=1720547749&GoogleAccessId=cambai-deployment%40camb-ai-82e9a.iam.gserviceaccount.com&Signature=AFJ7meODdqZgMVlj9JJiA%2BGUeYYSTFeL1h6Gzdg%2FjCfjzxKV3LBgd5xQDQUA1IONWLrVx%2FjMwtJCEK5xgmflvjPAn%2F1u8%2Fyr209a8pe%2Fr3cT%2F%2FATGJrj0GDt%2B3iz%2Fx8zQp6LFw4AgSq3tQTs%2BihHFfZXgG8mXxt4Gzi63%2B7mzMudYFoPfBp%2FeJUSR1ZcxBxh1Q78XUwYThE4kdz7IcQWO5bH68SDg91lkRdf6cFihbuk1FyoGzgaSXNnXkau%2F%2BcHEynjZNwRpkrBR21Uy0N71uk0HWe7KM0%2BxDHWvuUNVt2vhQV4oRestPgwJogyab8HGijIUJhEi393F%2BfzkU%2BvgQ%3D%3D'

In [ ]:
dubbed_info["transcript"]